In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 10:41:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from env import host, user, password

def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'


# These exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.

# Read the cases, department, and source data into their own Spark dataframes.

In [3]:
query = """SELECT * FROM cases"""
url = get_db_url("311_data")
cases_df = pd.read_sql(query, url)
cases_df = spark.createDataFrame(cases_df)
cases_df.show(4)

23/08/23 10:45:23 WARN TaskSetManager: Stage 0 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/08/23 10:45:28 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO| -2.012604167|        YES|     Storm Water|Removal Of Obstru...|4.322222222|     Closed| svcCRMSS|2215  GOL

In [4]:
query = """SELECT * FROM dept"""
url = get_db_url("311_data")
dep_df = pd.read_sql(query, url)
dep_df = spark.createDataFrame(dep_df)
dep_df.show(4)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 4 rows



In [5]:
query = """SELECT * FROM source"""
url = get_db_url("311_data")
source_df = pd.read_sql(query, url)
source_df = spark.createDataFrame(source_df)
source_df.show(4)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
|    3|   119403|  Betty De Hoyos|
+-----+---------+----------------+
only showing top 4 rows



# Let's see how writing to the local disk works in Spark:

## Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [6]:
(
    source_df.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("source_csv")
)

In [7]:
(
    source_df.write.format("json")
    .mode("overwrite")
    .option("header", "true")
    .save("source_json")
)

In [8]:
(
    dep_df.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("dept_csv")
)

(
    dep_df.write.format("json")
    .mode("overwrite")
    .option("header", "true")
    .save("dept_json")
)

In [9]:
(
    cases_df.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("cases_csv")
)

(
    cases_df.write.format("json")
    .mode("overwrite")
    .option("header", "true")
    .save("cases_json")
)

23/08/23 10:45:38 WARN TaskSetManager: Stage 8 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/08/23 10:45:46 WARN TaskSetManager: Stage 9 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


## Inspect your folder structure. What do you notice?

#### inside each folder is "parts" of the data

## Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [10]:
source_df.schema

StructType([StructField('index', LongType(), True), StructField('source_id', StringType(), True), StructField('source_username', StringType(), True)])

In [11]:
dep_df.schema

StructType([StructField('dept_division', StringType(), True), StructField('dept_name', StringType(), True), StructField('standardized_dept_name', StringType(), True), StructField('dept_subject_to_SLA', StringType(), True)])

In [12]:
cases_df.schema

StructType([StructField('case_id', LongType(), True), StructField('case_opened_date', StringType(), True), StructField('case_closed_date', StringType(), True), StructField('SLA_due_date', StringType(), True), StructField('case_late', StringType(), True), StructField('num_days_late', DoubleType(), True), StructField('case_closed', StringType(), True), StructField('dept_division', StringType(), True), StructField('service_request_type', StringType(), True), StructField('SLA_days', DoubleType(), True), StructField('case_status', StringType(), True), StructField('source_id', StringType(), True), StructField('request_address', StringType(), True), StructField('council_district', LongType(), True)])

In [13]:
cases_df.select("case_opened_date", "case_closed_date").show(5)

fmt = "M/d/yy H:mm"
cases_df = (
    cases_df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)


cases_df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

23/08/23 10:45:49 WARN TaskSetManager: Stage 10 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/08/23 10:45:53 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 10 (TID 52): Attempting to kill Python Worker


+----------------+----------------+
|case_opened_date|case_closed_date|
+----------------+----------------+
|     1/1/18 0:42|    1/1/18 12:29|
|     1/1/18 0:46|     1/3/18 8:11|
|     1/1/18 0:48|     1/2/18 7:57|
|     1/1/18 1:29|     1/2/18 8:13|
|     1/1/18 1:34|    1/1/18 13:29|
+----------------+----------------+
only showing top 5 rows



23/08/23 10:45:53 WARN TaskSetManager: Stage 11 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



23/08/23 10:45:57 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 11 (TID 53): Attempting to kill Python Worker


In [14]:
cases_df = cases_df.withColumn("case_closed", col("case_closed").cast("boolean"))

# Data Exploratory Questions

## How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [15]:
cases_df = (
    cases_df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

cases_df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

cases_df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)


23/08/23 10:45:58 WARN TaskSetManager: Stage 12 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/08/23 10:46:02 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 12 (TID 54): Attempting to kill Python Worker


+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 00:42:00|    2060|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-01 00:46:00|    2060|             0|            0|
|       true|2018-01-01 00:48:00|2018-01-01 00:48:00|    2060|             0|            0|
|       true|2018-01-01 01:29:00|2018-01-01 01:29:00|    2060|             0|            0|
|       true|2018-01-01 01:34:00|2018-01-01 01:34:00|    2060|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows



23/08/23 10:46:02 WARN TaskSetManager: Stage 13 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|      false|2018-01-02 09:39:00|2018-01-02 09:39:00|    2059|             0|         2059|
|      false|2018-01-02 10:49:00|2018-01-02 10:49:00|    2059|             0|         2059|
|      false|2018-01-02 13:45:00|2018-01-02 13:45:00|    2059|             0|         2059|
|      false|2018-01-02 14:09:00|2018-01-02 14:09:00|    2059|             0|         2059|
|      false|2018-01-02 14:34:00|2018-01-02 14:34:00|    2059|             0|         2059|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows



23/08/23 10:46:06 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 13 (TID 55): Attempting to kill Python Worker


#### Latest open case

In [16]:
latest_open_case = cases_df.filter(col("case_closed") == "false") \
                          .orderBy(col("case_opened_date").desc()) \
                          .first()
latest_open_case_pandas = [latest_open_case]

latest_open_case_df = spark.createDataFrame(latest_open_case_pandas).toPandas()
latest_open_case_df

23/08/23 10:46:07 WARN TaskSetManager: Stage 14 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,case_status,source_id,request_address,council_district,case_due_date,case_age,days_to_closed,case_lifetime
0,1014759619,2018-08-08 10:38:00,2018-08-08 10:38:00,8/22/18 10:38,NO,-13.443056,False,Code Enforcement,Bandit Signs,14.0,Open,CRM_Listener,"1935 MILITARY DR SW, San Antonio, 78221",3,2018-08-08 10:38:00,1841,0,1841


#### longest open case age

In [17]:
max_case_age_df = cases_df.groupBy("case_closed").agg(max(col("case_age")).alias("max_case_age"))
max_case_age_df.show()

23/08/23 10:46:15 WARN TaskSetManager: Stage 16 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-----------+------------+
|case_closed|max_case_age|
+-----------+------------+
|       true|        2425|
|      false|        2425|
+-----------+------------+



## How many Stray Animal cases are there?

In [18]:
type_counts = cases_df.groupBy('service_request_type').agg(col("service_request_type") == "Stray Animal").count()
type_counts

23/08/23 10:46:18 WARN TaskSetManager: Stage 19 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


335

## How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [19]:
filtered_cases = cases_df.filter((col("dept_division") == "Field Operations") & (col("service_request_type") != "Office Standby"))
filtered_cases.count()

23/08/23 10:46:21 WARN TaskSetManager: Stage 25 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


116915

## Convert the council_district column to a string column.

In [20]:
cases_df = cases_df.withColumn("council_district", col("council_district").cast("string"))

## Extract the year from the case_closed_date column.

In [21]:
cases_df = cases_df.withColumn("year", regexp_extract("case_closed_date", r"\d{4}", 0))

cases_df.select("year").show(5)

23/08/23 10:46:23 WARN TaskSetManager: Stage 28 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 5 rows



23/08/23 10:46:27 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 28 (TID 100): Attempting to kill Python Worker


In [22]:
year_counts = cases_df.groupBy("year").count()

year_counts.show()

23/08/23 10:46:27 WARN TaskSetManager: Stage 29 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----+------+
|year| count|
+----+------+
|2018|323370|
|2017|518334|
+----+------+



## Convert num_days_late from days to hours in a new column num_hours_late.

In [23]:
cases_df = cases_df.withColumn(
     "num_hours_late",
    (col("num_days_late") * 24)
)

In [24]:
cases_df.groupBy('num_hours_late').count()

DataFrame[num_hours_late: double, count: bigint]

In [25]:
cases_df.select("num_hours_late").show(5)

23/08/23 10:46:31 WARN TaskSetManager: Stage 32 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+
|     num_hours_late|
+-------------------+
|-23964.210278399998|
|      -48.302500008|
|      -72.536111112|
|      -360.27555552|
|        8.931944448|
+-------------------+
only showing top 5 rows



23/08/23 10:46:35 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 32 (TID 110): Attempting to kill Python Worker


## Join the cases data with the source and department data.

## Are there any cases that do not have a request source?

## What are the top 10 service request types in terms of number of requests?

## What are the top 10 service request types in terms of average days late?

## Does number of days late depend on department? (Answer without the use of a stats test)

## How do number of days late depend on department and request type?